In [1]:
#!pip install plotly

## Imports

In [2]:
# To convert to json objects
import json

# For Http calls
import requests

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# Date calculations
from datetime import datetime, timedelta

# For dataframe
import pandas as pd

# Plotting
import plotly.graph_objects as go

## Load environment variables

In [3]:
dotenv_path = Path('.env/graph')
load_dotenv(dotenv_path=dotenv_path)
GRAPH_API_KEY = os.getenv('GRAPH_API_KEY')

## Constants

In [4]:
# The graph.com endpoint
GRAPHQL_ENDPOINT = 'https://gateway.thegraph.com/api/{api_key}/\
subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV'.format(api_key=GRAPH_API_KEY)

# Token 1 is WETH
TOKEN1 = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

# Token 2 is WBTC
TOKEN2 = '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'

# How many days back from today's date
DAYS = 60

# How many hours back from the current time - only needed for hour prices
HOURS = 48

# Timeout in seconds
TIMEOUT = 40

# Query to get token info
TOKEN_QUERY = """
{{
  token(id: "{token}") {{
    name
    symbol
  }}
}}
"""

# Query to get token day data
TOKEN_DATA_DATA_QUERY = """
{{
  tokenDayDatas(
    where: {{token_: {{symbol: "{symbol}"}}, date_gt: {start}}}
    orderBy: date
  ) {{
    priceUSD
    date
  }}
}}
"""

In [5]:
def get_token_info(token:str) -> (str, str):
    """ Returns token symbol and name as a tuple
    Parameters:
    token : str
        token address
        
    Returns:
    tuple
        Token symbol and name as a tuple
    """
    query = TOKEN_QUERY.format(token=token)
    json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)
    token_symbol = json_data['data']['token']['symbol']
    token_name = json_data['data']['token']['name']
    return (token_symbol, token_name)

## Get Name and Symbol

In [6]:
# Holds token info for both tokens
token_info = {'token1': {}, 'token2': {}}

# Get token info for TOKEN1
symbol, name = get_token_info(token=TOKEN1)
token_info['token1'] = {'symbol': symbol, 'name': name}
# Get token info for TOKEN2
symbol, name = get_token_info(token=TOKEN2)
token_info['token2'] = {'symbol': symbol, 'name': name}

In [7]:
def get_daily_token_data(symbol:str, start:int) -> pd.DataFrame:
    """ Returns token daily data a dataframe with date and prices in USD
    Parameters:
    symbol : str
        token symbol
    start: int
        start time (unix epoch time)
        
    Returns:
    pd.DataFrame
        A dataframe consists of date and prices
    """   
    query = TOKEN_DATA_DATA_QUERY.format(symbol=symbol, start=start)
    json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)
    # Construct a DF from the tokenDayDatas which is an array consisting of date, fees and price
    df = pd.DataFrame(json_data['data']['tokenDayDatas'])
    # Convert fees and prices to floats
    df = df.astype({'priceUSD':'float'})
    # Remove any elemets with zero prices - some are duplicates with zero values
    df = df.query('priceUSD >0')
    # Covert to datetime object, pass 's' (seconds) for unit 
    df['date'] = pd.to_datetime(df['date'],unit='s')
    df.round({'priceUSD': 2})
    return df    

## Token1 daily data

In [8]:
# This is the starting date (DAYS befoe the current date)
period_start = int((datetime.today() - timedelta(days=DAYS)).timestamp())
token1_df = get_daily_token_data(symbol=token_info['token1']['symbol'], start=period_start)

# Set the date as the index
token1_df = token1_df.set_index('date')

## Plot daily prices

In [9]:
def plot_prices(token_data:dict, df:pd.DataFrame, hourly:bool=False) -> go.Figure:
    """ Returns the go.Figure created using DF
    Parameters:
    token_data : dict
        a dictionary containing token symbol and name
    df: pd.DataFrame
        DF containing data to plot
    hourly:bool
        True if we are plooting hourly prices or else we plot daily prices
        
    Returns:
    go.Figure:
        A go.Figure created from DF passed as a parameter
    """       
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['priceUSD'], mode='lines',
                             line_shape='spline', name=token_data['symbol']))

    # Adjust the title for hourly/daily
    if hourly:
        title = '{name} hourly prices for the past {period} hours'.format(
            name=token_data['name'], period=HOURS)
    else:
        title = '{name} daily prices for the past {period} days'.format(
            name=token_data['name'], period=DAYS)
        
    # Add title, axis labels etc.
    fig.update_layout(title=dict(text=title), xaxis=dict(title=dict(text='Date')),
        yaxis=dict(title=dict(text='Prices')),template='seaborn')
    return fig

## Plot Token1 Prices

In [17]:
plot_prices(token_data=token_info['token1'], df=token1_df).show()

## Token hourly data

In [11]:
def plot_hourly_token_prices():
    # This is the starting time, HOURS before now
    period_start = int((datetime.now() - timedelta(hours=HOURS)).timestamp())
    
    query = """
    {{
      tokenHourDatas(
        where: {{token_: {{symbol: "{symbol}"}}, periodStartUnix_gte: {start}}}
        orderBy: periodStartUnix
      ) {{
        priceUSD
        feesUSD
        periodStartUnix
      }}
    }}
    """
    query = query.format(symbol=token_info['token1']['symbol'], start=period_start)
    # Create a json object from the response
    json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)
    # Construct a DF from the tokenDayDatas which is an array consisting of date, fees and price
    df = pd.DataFrame(json_data['data']['tokenHourDatas'])
    # Convert fees and prices to floats
    df = df.astype({'priceUSD':'float'})
    # Remove any elemets with zero prices - some are duplicates with zero values
    df = df.query('priceUSD >0')
    # Covert to datetime object, pass 's' (seconds) for unit 
    df['date'] = pd.to_datetime(df['periodStartUnix'],unit='s')
    # Set the date as the index
    df = df.set_index('date')
    df.round({'priceUSD': 2})
    plot_prices(token_data=token_info['token1'], df=df, hourly=True).show()

## Plot hourly prices

In [18]:
plot_hourly_token_prices()

## 7D MA

In [20]:
# 7 day MA
MA7_series = token1_df['priceUSD'].rolling(window=7).mean()
MA7_df = MA7_series.to_frame()
# Only include non nan values
MA7_df = MA7_df[MA7_df['priceUSD'].notna()]

fig = plot_prices(token_data=token_info['token1'], df=token1_df)
fig.add_trace(go.Scatter(x=MA7_df.index, y=MA7_df['priceUSD'], mode='lines',
                         line_shape='spline', name='7 day MA for {symbol}'.format(
                             symbol=token_info['token1']['symbol'])))


## Token2 daily data

In [19]:
token2_df = get_daily_token_data(symbol=token_info['token2']['symbol'], start=period_start)
token2_df = token2_df.set_index('date')
# Plot Token2 prices
plot_prices(token_data=token_info['token2'], df=token2_df).show()

## Token 1 to Token 2 ratio

In [21]:
tk1_tk2_ratio_series = token1_df['priceUSD']/token2_df['priceUSD']
tk1_tk2_ratio_df = tk1_tk2_ratio_series.to_frame()

fig = go.Figure()
fig.add_trace(go.Scatter(x=tk1_tk2_ratio_df.index,
                         y=tk1_tk2_ratio_df['priceUSD'], mode='lines', line_shape='spline'))

token1_data = token_info['token1']
token2_data = token_info['token2']

# Edit the layout
fig.update_layout(
    title=dict(text='{token1} to {token2} ratio past {days} days'.format(
        token1=token1_data['name'], token2=token2_data['name'], days=DAYS)),
    xaxis=dict(title=dict(text='Date')),
    yaxis=dict(title=dict(text='{token1} / {token2} ratio'.format(
        token1=token1_data['symbol'], token2=token2_data['symbol']))),
    template='seaborn')